<a href="https://colab.research.google.com/github/20911357Pinyaphat/smart-finance-assistant/blob/main/smart-finance-girly-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas matplotlib gradio hands-on-ai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import os
from getpass import getpass

# Configure hands-on-ai server connection
os.environ['HANDS_ON_AI_SERVER'] = 'https://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'llama3.2'
os.environ['HANDS_ON_AI_API_KEY'] = getpass('Enter your API key: ')

print("🔑 Hands-on-AI configured successfully!")

Enter your API key: ··········
🔑 Hands-on-AI configured successfully!


'''
This section explains what the code does.
It’s clean, readable, and perfect for Colab notebooks.
'''
'''
# 💖 **Budget Babe: Smart Finance Assistant**

Welcome to Budget Babe — your friendly, empowering finance assistant designed to help users in Australia track their recurring subscriptions, understand their spending habits, and reach their savings goals with confidence.

🎯 **What This App Is For:**
- Load real data from `sample_transactions.csv` that includes popular Australian subscriptions
- Automatically categorize each subscription (e.g., Streaming, Music, Food, Utilities)
- Help users identify duplicate or overlapping services they might want to cancel
- Let users name themselves and their chatbot for a personalized experience
- Allow users to set a savings goal name (e.g., "Trip to Bali", "New Laptop") and track progress
- Show monthly and yearly spending reports
- Display infographics and graphs to visualize spending by category
- Provide friendly, girly, and empowering chatbot advice to help users make smart decisions

✨ **Why We're Building This:**
Budgeting should feel joyful, intuitive, and supportive — not boring or stressful. Budget Babe is here to:
- Make subscription tracking effortless
- Help users save money without guilt
- Celebrate every step toward financial freedom

Let’s glow up your budget together 💅
'''


In [3]:
import gradio as gr
print(gr.__version__)


5.49.0


In [4]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt

'''
we import Gradio for the UI, pandas for data manipulation, and matplotlib for plotting
'''

# 📂 Load and clean data from GitHub
csv_url = "https://raw.githubusercontent.com/20911357Pinyaphat/smart-finance-assistant/main/data/sample_transactions.csv"
df = pd.read_csv(csv_url)
df.columns = df.columns.str.strip().str.title()
df["Amount"] = pd.to_numeric(df["Amount"].replace(r'[\$,]', '', regex=True), errors="coerce").fillna(0)

# ✨ Auto-categorize subscriptions
def categorize(description):
    desc = description.lower()
    if any(x in desc for x in ["spotify", "apple music", "tidal", "youtube music"]):
        return "Music"
    elif any(x in desc for x in ["netflix", "disney", "stan", "binge", "foxtel", "paramount", "prime", "youtube premium"]):
        return "Movies & TV"
    elif any(x in desc for x in ["gym", "fitness", "classpass", "anytime fitness"]):
        return "Fitness"
    elif any(x in desc for x in ["audible", "kindle", "scribd"]):
        return "Reading"
    elif any(x in desc for x in ["hellofresh", "marley spoon"]):
        return "Food"
    elif any(x in desc for x in ["canva", "adobe", "microsoft", "notion"]):
        return "Software"
    elif any(x in desc for x in ["nbn", "optus", "telstra", "vodafone"]):
        return "Internet & Phone"
    elif any(x in desc for x in ["uber", "opal", "transperth"]):
        return "Transport"
    elif any(x in desc for x in ["savings", "goal deposit"]):
        return "Savings"
    else:
        return "Other"

df["Category"] = df["Description"].apply(categorize)
df = df.dropna(subset=["Description", "Category"])
services = df["Description"].unique().tolist()
''' defining a function to categorise each subscription based on keywords which helps group services into meaningful spending categories for analysis'''
# 🎨 Group services by category
category_map = {}
for _, row in df.iterrows():
    cat = row["Category"]
    service = row["Description"]
    category_map.setdefault(cat, []).append(service)
    '''
    dictionary mapping for each category to its list of services'''

# 💖 Store user info
user_info = {}
'''
We use a global dictionary to store user-specific data like name, goal, and target amount.
This allows personalized responses throughout the app.
'''
# ✨ Helper functions
def save_user_info(name, bot_name, goal_name, goal_amount):
    user_info["name"] = name
    user_info["bot"] = bot_name
    user_info["goal"] = goal_name
    user_info["target"] = float(goal_amount) if goal_amount else 0
    return f"Hi {name}! 💖 Your budgeting assistant '{bot_name}' is ready to help you save for '{goal_name}' (${goal_amount})!"


def collect_selected(*args):
    selected = []
    for group in args:
        if group:
            selected.extend(group)
    return selected

def calculate_spending(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    yearly = total * 12
    breakdown = selected_df.groupby("Category")["Amount"].sum().to_dict()

    duplicates = []
    if sum([s in selected_services for s in ["Spotify", "Apple Music"]]) > 1:
        duplicates.append("🎧 Multiple music platforms (Spotify + Apple Music)")
    if sum([s in selected_services for s in ["Netflix", "Disney+", "Stan", "Binge", "Prime Video"]]) > 2:
        duplicates.append("📺 Lots of streaming services — consider cutting one")

    result = f"💖 Total Monthly Spend: ${total:.2f}\n"
    result += f"📅 Estimated Yearly Spend: ${yearly:.2f}\n\n"
    for cat, amt in breakdown.items():
        result += f"• {cat}: ${amt:.2f}\n"
    if duplicates:
        result += "\n⚠️ Possible Duplicates:\n" + "\n".join(duplicates)

    target = user_info.get("target", 0)
    if target > 0:
        monthly_savings_needed = target / 12
        result += f"\n🎯 You need to save ${monthly_savings_needed:.2f}/month to reach your goal.\n"
        if total > monthly_savings_needed:
            result += "💸 Consider trimming subscriptions to meet your savings target!"
        else:
            result += "✅ You're on track to hit your goal — amazing!"

    return result

def budget_babe_chat(user_message, selected_services, avatar):
    if not selected_services:
        return "⚠️ No subscriptions selected. Pick a few and try again 💅"

    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    category_totals = selected_df.groupby("Category")["Amount"].sum().to_dict()

    name = user_info.get("name", "babe")
    goal = user_info.get("goal", "your goal")
    target = user_info.get("target", 0)

    # 🎀 Tone logic
    if avatar == "Cheeky Babe":
        style = lambda msg: f"{msg} 💅"
    elif avatar == "Zen Coach":
        style = lambda msg: f"{msg} 🧘 Deep breaths and smart choices."
    elif avatar == "Money Nerd":
        style = lambda msg: f"{msg} 📊 Data-driven and fabulous."
    else:
        style = lambda msg: msg

    # 💬 Response logic
    response = f"Hey {name}, you're spending ${total:.2f}/month.\n"

    if "streaming" in user_message.lower():
        streaming = category_totals.get("Movies & TV", 0)
        response += f"📺 Streaming total: ${streaming:.2f}\n"
        if streaming > 30:
            response += style("💸 That’s a lot of screen time! Maybe cancel one or two platforms?")
        else:
            response += style("✅ Streaming is under control — love that for you!")
    elif "music" in user_message.lower():
        music = category_totals.get("Music", 0)
        response += f"🎧 Music total: ${music:.2f}\n"
        if music > 15:
            response += style("🎶 Maybe pick one platform and save the rest for concert tickets!")
        else:
            response += style("🎵 Sounds like a sweet deal — rock on!")
    elif "goal" in user_message.lower():
        if target > 0:
            monthly_savings_needed = target / 12
            response += f"🎯 To reach '{goal}', you need to save ${monthly_savings_needed:.2f}/month.\n"
            if total > monthly_savings_needed:
                response += style("💸 Let’s trim a few subscriptions and get you there faster!")
            else:
                response += style("✅ You’re on track — keep it up!")
        else:
            response += style("🎯 Set a target amount to get personalized savings advice!")
    else:
        response += style("💡 Try asking about streaming, music, or your savings goal!")

    return response

def styled_chat_response(user_message, selected_services, avatar):
    raw = budget_babe_chat(user_message, selected_services, avatar)
    color = "#d63384" if avatar == "Cheeky Babe" else "#3399ff" if avatar == "Zen Coach" else "#28a745"
    return f"<div style='background-color:{color};color:white;padding:10px;border-radius:10px;'>{raw}</div>"

def spending_emoji(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    if total > 150:
        return "💸 Oof! That’s a lot of sparkle — time to trim?"
    elif total > 80:
        return "✨ Balanced but could be shinier!"
    else:
        return "🌈 Budget Babe approved — keep glowing!"
def sparkle_rating(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    score = max(0, 100 - int(total))  # Simple inverse score

    if score >= 90:
        badge = "🌟 Budget Babe Elite"
    elif score >= 70:
        badge = "💖 Smart Saver"
    else:
        badge = "💸 Needs a glow-up"

    return f"### Your Sparkle Score: {score}/100\n{badge}"
def savings_progress(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    target = user_info.get("target", 0)
    if target == 0:
        return 0, "⚠️ Set a savings goal to track progress 💅"

    monthly_savings_needed = target / 12
    progress = max(0, min(100, int((monthly_savings_needed - total) / monthly_savings_needed * 100)))

    if progress >= 100:
        return 100, "🎉 You did it! Goal reached — confetti time!"
    elif progress >= 80:
        return progress, "✨ Almost there — keep sparkling!"
    else:
        return progress, "💪 Keep going — every cut counts!"
def plot_spending(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    breakdown = selected_df.groupby("Category")["Amount"].sum()
    if breakdown.empty:
        return None
    fig, ax = plt.subplots()
    breakdown.plot(kind="bar", color="#d63384", ax=ax)
    ax.set_title("Monthly Spending by Category")
    ax.set_ylabel("Amount ($)")
    return fig

def suggest_cuts(selected_services):
    suggestions = []
    if sum([s in selected_services for s in ["Spotify", "Apple Music"]]) > 1:
        suggestions.append("🎧 Cut one music platform (Spotify or Apple Music)")
    if sum([s in selected_services for s in ["Netflix", "Disney+", "Stan", "Binge", "Prime Video"]]) > 2:
        suggestions.append("📺 Too many streaming services — cut one or two")
    selected_df = df[df["Description"].isin(selected_services)]
    if selected_df["Amount"].max() > 40:
        expensive = selected_df[selected_df["Amount"] > 40]["Description"].tolist()
        for item in expensive:
            suggestions.append(f"💸 Consider cutting or downgrading: {item}")
    return "\n".join(suggestions) if suggestions else "✅ No obvious cuts needed — you're doing great!"

def generate_report(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]
    total = selected_df["Amount"].sum()
    top = selected_df.groupby("Category")["Amount"].sum().sort_values(ascending=False)
    report = f"📋 Monthly Report\n\nTotal Spend: ${total:.2f}\n\nTop Categories:\n"
    for cat, amt in top.items():
        report += f"• {cat}: ${amt:.2f}\n"
    return report

def reset_all():
    user_info.clear()
    return "", "", "", "", "", "", "", None



import gradio as gr

# 🚀 Launch app with custom pink styling
with gr.Blocks() as app:
    # 🎀 Inject pink styles and fonts
    gr.HTML("""
    <link href="https://fonts.googleapis.com/css2?family=Quicksand&display=swap" rel="stylesheet">
    <style>
    body {
        background: linear-gradient(135deg, #ffe6f0, #fff0f5);
        font-family: 'Quicksand', sans-serif;
    }
    h2 {
        color: #d63384;
        font-size: 2em;
        text-align: center;
        margin-top: 20px;
    }
    button {
        background-color: #ff69b4 !important;
        color: white !important;
        border-radius: 12px !important;
        font-weight: bold;
    }
    input, textarea, select {
        border-radius: 10px !important;
        border: 2px solid #ffb6c1 !important;
    }
    </style>
    """)

    # 💖 App title
    gr.Markdown("<h2>💖 Budget Babe: Smart Finance Assistant</h2>")

    # Welcome
    with gr.Row():
        name_input = gr.Textbox(label="Your name")
        bot_name_input = gr.Textbox(label="Name your chatbot")
        goal_input = gr.Textbox(label="Your savings goal")
        goal_amount_input = gr.Textbox(label="Target amount ($)")
    welcome_output = gr.Textbox(label="Welcome Message", lines=3)
    gr.Button("Start 💅").click(
        save_user_info,
        inputs=[name_input, bot_name_input, goal_input, goal_amount_input],
        outputs=welcome_output
    )

    # Avatar picker
    gr.Markdown("### 🎀 Choose your chatbot personality")
    avatar_input = gr.Dropdown(label="Chatbot Style", choices=["Cheeky Babe", "Zen Coach", "Money Nerd"], value="Cheeky Babe")

    # Subscription picker
    gr.Markdown("### 📋 Pick your subscriptions by category")
    category_inputs = []
    with gr.Row():
        for cat, services in category_map.items():
            with gr.Column():
                emoji = "🎧" if "Music" in cat else "📺" if "Movies" in cat else "🏋️" if "Fitness" in cat else "📚" if "Reading" in cat else "🛍️" if "Shopping" in cat else ""
                category_inputs.append(
                    gr.CheckboxGroup(label=f"{emoji} {cat}", choices=services)
                )

    # Spending breakdown
    spending_output = gr.Textbox(label="Spending Breakdown", lines=8)
    gr.Button("Calculate Spending").click(
        fn=lambda *args: calculate_spending(collect_selected(*args)),
        inputs=category_inputs,
        outputs=spending_output
    )

    # Chatbot advice
    user_message = gr.Textbox(label="💬 Ask Budget Babe", placeholder="e.g. How much do I spend on streaming?")
    chatbot_output = gr.Textbox(label="Chatbot Response", lines=6)
    gr.Button("Get Advice").click(
        fn=lambda msg, avatar, *args: budget_babe_chat(msg, collect_selected(*args), avatar),
        inputs=[user_message, avatar_input] + category_inputs,
        outputs=chatbot_output
    )

    # Merged graph + report + sparkle score
    gr.Markdown("### 📊 Monthly Spending Overview")
    graph_output = gr.Plot()
    report_output = gr.Textbox(label="Summary", lines=6)
    sparkle_score = gr.Markdown("")  # 🌟 Sparkle score badge
    gr.Button("Show Graph + Summary").click(
        fn=lambda *args: (
            plot_spending(collect_selected(*args)),
            generate_report(collect_selected(*args)),
            sparkle_rating(collect_selected(*args))
        ),
        inputs=category_inputs,
        outputs=[graph_output, report_output, sparkle_score]
    )

    # Cut suggestions
    cut_output = gr.Textbox(label="Cut Suggestions ✂️", lines=6)
    gr.Button("Suggest Cuts").click(
        fn=lambda *args: suggest_cuts(collect_selected(*args)),
        inputs=category_inputs,
        outputs=cut_output
    )

    # Reset button
    gr.Button("Reset All 🔄").click(
        fn=reset_all,
        inputs=[],
        outputs=[
            name_input, bot_name_input, goal_input, goal_amount_input,
            welcome_output, spending_output, chatbot_output, report_output
        ]
    )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0161f9f3af32588178.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
